In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay, f1_score
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_x = train.drop(columns=['even','label'])
train_y = train['label']
val_x = val.drop(columns=['even','label'])
val_y = val['label']

In [ ]:
train_x = train_x.to_numpy().astype(np.float64)/255.0
train_y = train_y.to_numpy()
val_x = val_x.to_numpy().astype(np.float64)/255.0
val_y= val_y.to_numpy()

In [ ]:
# XGBoost with n = 100 and max_depth = 5
# time taken = approx. 10 min

In [ ]:
class tree_node:
    def __init__(self, feature_idx=None, threshold=None, left_node=None, right_node=None, leaf_value=None):
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.left_node = left_node
        self.right_node = right_node
        self.leaf_value = leaf_value

class XGBoostMultiClass:
    def __init__(self, num_classes, n_estimators=200, learning_rate=0.2, max_depth=5):
        self.K = num_classes
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.init_scores = None

    def softmax(self, logits):
        logits = logits - np.max(logits, axis=1, keepdims=True)
        exp_vals = np.exp(logits)
        return exp_vals / np.sum(exp_vals, axis=1, keepdims=True)

    def build(self, X, grad, hess, depth):
        if depth >= self.max_depth or len(X) == 0:
            leaf_value = -np.sum(grad) / (np.sum(hess) + 1e-8)
            return tree_node(leaf_value=leaf_value)

        G_total, H_total = np.sum(grad), np.sum(hess)
        best_gain = -float('inf')
        best_feat = None
        best_thresh = None
        best_left = best_right = None

        feature_count = int(np.sqrt(X.shape[1])) + 1
        feature_indices = np.random.choice(X.shape[1], feature_count, replace=False)

        for j in feature_indices:
            values = np.unique(X[:, j])
            thresholds = np.random.choice(values, min(10, len(values)), replace=False)

            for threshold in thresholds:
                left = X[:, j] <= threshold
                right = ~left

                if not np.any(left) or not np.any(right):
                    continue

                G_l, H_l = np.sum(grad[left]), np.sum(hess[left])
                G_r, H_r = np.sum(grad[right]), np.sum(hess[right])

                gain = 0.5 * (
                    G_l**2 / (H_l + 1e-8) +
                    G_r**2 / (H_r + 1e-8) -
                    G_total**2 / (H_total + 1e-8)
                )

                if gain > best_gain:
                    best_gain = gain
                    best_feat = j
                    best_thresh = threshold
                    best_left = left
                    best_right = right

        if best_gain == -float('inf'):
            leaf_value = -np.sum(grad) / (np.sum(hess) + 1e-8)
            return tree_node(leaf_value=leaf_value)

        left_node = self.build(X[best_left],  grad[best_left],  hess[best_left],  depth+1)
        right_node = self.build(X[best_right], grad[best_right], hess[best_right], depth+1)

        return tree_node(feature_idx=best_feat, threshold=best_thresh,
                         left_node=left_node, right_node=right_node)

    def pred_one(self, x, node):
        while node.leaf_value is None:
            if x[node.feature_idx] <= node.threshold:
                node = node.left_node
            else:
                node = node.right_node
        return node.leaf_value
    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        N = len(y)

        Y = np.eye(self.K)[y]

        scores = np.zeros((N, self.K))
        self.init_scores = np.zeros(self.K)

        self.trees = []

        for m in range(self.n_estimators):
            probs = self.softmax(scores)
            grad = probs - Y
            hess = probs * (1 - probs)

            round_trees = []
            for k in range(self.K):
                tree = self.build(X, grad[:, k], hess[:, k], depth=0)
                update = np.array([self.pred_one(row, tree) for row in X])
                scores[:, k] += self.learning_rate * update

                round_trees.append(tree)

            self.trees.append(round_trees)
    def predict(self, X):
        X = np.asarray(X)
        N = X.shape[0]

        scores = np.zeros((N, self.K))

        for round_trees in self.trees:
            for k in range(self.K):
                update = np.array([self.pred_one(row, round_trees[k]) for row in X])
                scores[:, k] += self.learning_rate * update

        probs = self.softmax(scores)
        return np.argmax(probs, axis=1)

    def predict_proba(self, X):
        X = np.asarray(X)
        N = X.shape[0]

        scores = np.zeros((N, self.K))

        for round_trees in self.trees:
            for k in range(self.K):
                update = np.array([self.pred_one(row, round_trees[k]) for row in X])
                scores[:, k] += self.learning_rate * update

        return self.softmax(scores)

In [ ]:
model = XGBoostMultiClass(num_classes=10, n_estimators=100, learning_rate=0.2, max_depth=5)
starttime = time.time()
model.fit(train_x, train_y)
ypred = model.predict(val_x)
print("Time taken:", time.time() - starttime)
print("Accuracy:", accuracy_score(val_y, ypred))
print("F1 Score:", f1_score(val_y, ypred, average='macro'))